In [8]:
import pandas as pd 

df = pd.read_csv("../data/raw_data_orders.csv")

df.head(1)

pd.set_option("display.max_colwidth", None)

In [9]:
'''Functions'''


import re 

def normalize_text(s):
    if pd.isna(s):
        return ""
    s = str(s).lower()
    s=re.sub(r"\s+", " ", s)
    return s.strip()

def split_products(s):
    if not s:
        return []
    
    # 1) insert a separator AFTER "(xN)" if another letter starts right away
    #    e.g. "... (x1)rurka ..." -> "... (x1) | rurka ..."
    s = re.sub(r"(\(x\d+\))(?=\S)", r"\1 | ", s)

    # 2) now split on " + ", on "zestaw" starts, and on our inserted separator
    parts = re.split(r"\s\+\s|\s\|\s|(?=\bzestaw\b)", s)

    return [p.strip() for p in parts if p.strip()]



import re

def get_qty(s):
    m = re.search(r"\(x(\d+)\)", s)
    return int(m.group(1)) if m else 1




def strip_qty(s):
    return re.sub(r"\s*\(x\d+\)\s*", " ", s).strip()



In [6]:
'''MAP'''

RULES = {
    "bottle": [r"\bbutelka\b"],
    "pitcher": [r"\bdzbanek\b"],
    "accessory": [r"\brurka\b", r"\bnakretka\b", r"\bnakrętka\b"],
    "filter_set": [r"\bzestaw\b.*\bfiltr", r"\bzestaw\b.*\bwklad", r"\bzestaw\b.*\bwkład"],
    "filter": [r"\bfiltr\b", r"\bwklad\b", r"\bwkład\b"],
}

RULES["filter"] = [r"\bfiltr\b", r"\bfiltry\b", r"\bwklad\b", r"\bwkład\b", r"\bwklady\b", r"\bwkłady\b"]


In [10]:
CATEGORY_ORDER = ["bottle", "pitcher", "accessory", "filter_set", "filter"]

def classify(text):
    for cat in CATEGORY_ORDER:
        patterns = RULES[cat]
        if any(re.search(p, text) for p in patterns):
            return cat
    return "other"


def extract_pack_size(text):
    # looks for "3 filtry", "8 filtrów", "2 wkłady", etc.
    m = re.findall(r"\b(\d+)\b\s*(filtr|filtry|filtrow|filtrów|wklad|wkład|wklady|wkłady)", text)
    if m:
        return sum(int(n) for n, _ in m)

    # fallback: "zestaw 3 ..." (if sometimes missing 'filtry' right after)
    m2 = re.search(r"\bzestaw\b\s*(\d+)\b", text)
    if m2:
        return int(m2.group(1))

    return np.nan


def has_any(text, patterns):
    return any(re.search(p, text) for p in patterns)



In [12]:
import pandas as pd

# 1) keep only what we need for aggregation + decoding
df = df[["kolejności", "Anon", "Data zakupu", "Produkty"]].copy()

# 2) parse date (adjust dayfirst if needed)
df["Data zakupu"] = pd.to_datetime(df["Data zakupu"], errors="coerce")

# 3) normalized text column (uses your normalize_text function)
df["produkty_clean"] = df["Produkty"].apply(normalize_text)

# quick checks
print("rows:", len(df))
print("bad dates:", df["Data zakupu"].isna().sum())
df.head(3)


/var/folders/2w/84n_l6ld0kq5y9sp9jw4q50c0000gn/T/ipykernel_4183/2783820120.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Data zakupu"] = pd.to_datetime(df["Data zakupu"], errors="coerce")


rows: 16688
bad dates: 0


,kolejności,Anon,Data zakupu,Produkty,produkty_clean
0,68640194,ANON_0001,2022-11-09,"Butelka filtrująca Dafi SOLID 0,7 l szafirowa + filtr węglowy (x1)Rurka na filtr do butelki filtrującej Dafi SOLID 0,7 l szafirowym (x1)ZESTAW 3 filtry do butelki filtrującej Dafi SOFT i SOLID szafirowy (x1)","butelka filtrująca dafi solid 0,7 l szafirowa + filtr węglowy (x1)rurka na filtr do butelki filtrującej dafi solid 0,7 l szafirowym (x1)zestaw 3 filtry do butelki filtrującej dafi soft i solid szafirowy (x1)"
1,69067553,ANON_0002,2022-11-11,ZESTAW 2 filtry + nakrętka do butelki filtrującej Dafi SOFT miętowy (x1),zestaw 2 filtry + nakrętka do butelki filtrującej dafi soft miętowy (x1)
2,69571584,ANON_0003,2022-11-13,"ZESTAW 3 filtry do butelki filtrującej Dafi SOFT i SOLID cytrynowy (x1)Butelka filtrująca Dafi SOFT 0,3 l stalowa + filtr węglowy (x1)Butelka filtrująca Dafi SOFT 0,5 l jagodowa + filtr węglowy (x1)ZESTAW 3 filtry do butelki filtrującej Dafi SOFT i SOLID jeansowy (x1)ZESTAW filtrów Unimax: 2 standard + 2 magnezowe Mg+ do dzbanków Dafi (x1)","zestaw 3 filtry do butelki filtrującej dafi soft i solid cytrynowy (x1)butelka filtrująca dafi soft 0,3 l stalowa + filtr węglowy (x1)butelka filtrująca dafi soft 0,5 l jagodowa + filtr węglowy (x1)zestaw 3 filtry do butelki filtrującej dafi soft i solid jeansowy (x1)zestaw filtrów unimax: 2 standard + 2 magnezowe mg+ do dzbanków dafi (x1)"


In [13]:
df["product_parts"] = df["produkty_clean"].apply(split_products)


In [14]:
df[["produkty_clean", "product_parts"]].head(3)


,produkty_clean,product_parts
0,"butelka filtrująca dafi solid 0,7 l szafirowa + filtr węglowy (x1)rurka na filtr do butelki filtrującej dafi solid 0,7 l szafirowym (x1)zestaw 3 filtry do butelki filtrującej dafi soft i solid szafirowy (x1)","[butelka filtrująca dafi solid 0,7 l szafirowa, filtr węglowy (x1), rurka na filtr do butelki filtrującej dafi solid 0,7 l szafirowym (x1), zestaw 3 filtry do butelki filtrującej dafi soft i solid szafirowy (x1)]"
1,zestaw 2 filtry + nakrętka do butelki filtrującej dafi soft miętowy (x1),"[zestaw 2 filtry, nakrętka do butelki filtrującej dafi soft miętowy (x1)]"
2,"zestaw 3 filtry do butelki filtrującej dafi soft i solid cytrynowy (x1)butelka filtrująca dafi soft 0,3 l stalowa + filtr węglowy (x1)butelka filtrująca dafi soft 0,5 l jagodowa + filtr węglowy (x1)zestaw 3 filtry do butelki filtrującej dafi soft i solid jeansowy (x1)zestaw filtrów unimax: 2 standard + 2 magnezowe mg+ do dzbanków dafi (x1)","[zestaw 3 filtry do butelki filtrującej dafi soft i solid cytrynowy (x1), butelka filtrująca dafi soft 0,3 l stalowa, filtr węglowy (x1), butelka filtrująca dafi soft 0,5 l jagodowa, filtr węglowy (x1), zestaw 3 filtry do butelki filtrującej dafi soft i solid jeansowy (x1), zestaw filtrów unimax: 2 standard, 2 magnezowe mg+ do dzbanków dafi (x1)]"


In [15]:
parts = (
    df[["kolejności", "Anon", "Data zakupu", "product_parts"]]
      .explode("product_parts")
      .rename(columns={"product_parts": "part_raw"})
      .copy()
)

parts["part_raw"] = parts["part_raw"].fillna("")


In [16]:
print("orders:", df.shape[0])
print("line items:", parts.shape[0])

parts.head(10)


orders: 16688
line items: 40429


,kolejności,Anon,Data zakupu,part_raw
0,68640194,ANON_0001,2022-11-09,"butelka filtrująca dafi solid 0,7 l szafirowa"
0,68640194,ANON_0001,2022-11-09,filtr węglowy (x1)
0,68640194,ANON_0001,2022-11-09,"rurka na filtr do butelki filtrującej dafi solid 0,7 l szafirowym (x1)"
0,68640194,ANON_0001,2022-11-09,zestaw 3 filtry do butelki filtrującej dafi soft i solid szafirowy (x1)
1,69067553,ANON_0002,2022-11-11,zestaw 2 filtry
1,69067553,ANON_0002,2022-11-11,nakrętka do butelki filtrującej dafi soft miętowy (x1)
2,69571584,ANON_0003,2022-11-13,zestaw 3 filtry do butelki filtrującej dafi soft i solid cytrynowy (x1)
2,69571584,ANON_0003,2022-11-13,"butelka filtrująca dafi soft 0,3 l stalowa"
2,69571584,ANON_0003,2022-11-13,filtr węglowy (x1)
2,69571584,ANON_0003,2022-11-13,"butelka filtrująca dafi soft 0,5 l jagodowa"


In [17]:
parts["qty"] = parts["part_raw"].apply(get_qty)
parts["part_clean"] = parts["part_raw"].apply(strip_qty)


In [20]:
parts["category"] = parts["part_clean"].apply(classify)


In [21]:
parts["category"].value_counts()


category
filter        11504
bottle         8749
filter_set     7587
other          6646
pitcher        4959
accessory       984
Name: count, dtype: int64

In [24]:
import numpy as np

parts["pack_size"] = parts["part_clean"].apply(extract_pack_size)
parts["units"] = parts["pack_size"].fillna(1) * parts["qty"]


In [25]:
import re

parts["bottle_filter_units"] = 0
parts["pitcher_filter_units"] = 0
parts["bottle_included_filter_units"] = 0

is_consumable = parts["category"].isin(["filter", "filter_set"])

is_wklad = parts["part_clean"].str.contains(r"\bwkład\b|\bwkłady\b|\bwklady\b|\bwklad\b", regex=True)
is_filtr = parts["part_clean"].str.contains(r"\bfiltr\b|\bfiltry\b|\bfiltrów\b|\bfiltrow\b", regex=True)

mask_pitcher = is_consumable & is_wklad
mask_bottle  = is_consumable & is_filtr & (~is_wklad)

parts.loc[mask_pitcher, "pitcher_filter_units"] = parts.loc[mask_pitcher, "units"]
parts.loc[mask_bottle,  "bottle_filter_units"]  = parts.loc[mask_bottle,  "units"]

# bottle includes 1 filter
parts.loc[parts["category"].eq("bottle"), "bottle_included_filter_units"] = parts.loc[
    parts["category"].eq("bottle"), "qty"
]


In [26]:
parts.loc[
    parts["category"].isin(["filter","filter_set","bottle","accessory","pitcher"]),
    ["part_clean","category","qty","pack_size","units","bottle_filter_units","pitcher_filter_units","bottle_included_filter_units"]
].head(40)


,part_clean,category,qty,pack_size,units,bottle_filter_units,pitcher_filter_units,bottle_included_filter_units
0,"butelka filtrująca dafi solid 0,7 l szafirowa",bottle,1,NaN,1.0,0,0,1
0,filtr węglowy,filter,1,NaN,1.0,1,0,0
0,"rurka na filtr do butelki filtrującej dafi solid 0,7 l szafirowym",accessory,1,NaN,1.0,0,0,0
0,zestaw 3 filtry do butelki filtrującej dafi soft i solid szafirowy,filter_set,1,3.0,3.0,3,0,0
1,zestaw 2 filtry,filter_set,1,2.0,2.0,2,0,0
1,nakrętka do butelki filtrującej dafi soft miętowy,accessory,1,NaN,1.0,0,0,0
2,zestaw 3 filtry do butelki filtrującej dafi soft i solid cytrynowy,filter_set,1,3.0,3.0,3,0,0
2,"butelka filtrująca dafi soft 0,3 l stalowa",bottle,1,NaN,1.0,0,0,1
2,filtr węglowy,filter,1,NaN,1.0,1,0,0
2,"butelka filtrująca dafi soft 0,5 l jagodowa",bottle,1,NaN,1.0,0,0,1


In [27]:
parts["bottle_units"] = 0
parts["pitcher_units"] = 0
parts["accessory_units"] = 0
parts["other_units"] = 0

parts.loc[parts["category"].eq("bottle"), "bottle_units"] = parts.loc[parts["category"].eq("bottle"), "qty"]
parts.loc[parts["category"].eq("pitcher"), "pitcher_units"] = parts.loc[parts["category"].eq("pitcher"), "qty"]
parts.loc[parts["category"].eq("accessory"), "accessory_units"] = parts.loc[parts["category"].eq("accessory"), "qty"]
parts.loc[parts["category"].eq("other"), "other_units"] = parts.loc[parts["category"].eq("other"), "qty"]


In [28]:
ORDER_COLS = ["kolejności", "Anon", "Data zakupu"]

order_level = (
    parts.groupby(ORDER_COLS, as_index=False)[
        ["bottle_units","pitcher_units","accessory_units","other_units",
         "bottle_filter_units","pitcher_filter_units","bottle_included_filter_units"]
    ].sum()
)

order_level["total_bottle_filters"] = (
    order_level["bottle_filter_units"] + order_level["bottle_included_filter_units"]
)


In [29]:
order_level.head(20)


,kolejności,Anon,Data zakupu,bottle_units,pitcher_units,accessory_units,other_units,bottle_filter_units,pitcher_filter_units,bottle_included_filter_units,total_bottle_filters
0,68640194,ANON_0001,2022-11-09,1,0,1,0,4,0,1,5
1,69067553,ANON_0002,2022-11-11,0,0,1,0,2,0,0,2
2,69571584,ANON_0003,2022-11-13,2,0,0,1,9,0,2,11
3,69716225,ANON_0004,2022-11-14,0,1,0,0,0,2,0,0
4,69779965,ANON_0005,2022-11-14,0,1,0,0,0,0,0,0
5,69825101,ANON_0006,2022-11-14,0,1,0,0,3,8,0,3
6,69871741,ANON_0007,2022-11-14,1,0,0,0,4,0,1,5
7,69955393,ANON_0008,2022-11-15,0,1,0,1,0,0,0,0
8,70018996,ANON_0009,2022-11-15,0,1,0,1,0,0,0,0
9,70059452,ANON_0010,2022-11-15,1,0,0,0,1,0,1,2


In [30]:
order_level[["bottle_units","pitcher_units","total_bottle_filters","pitcher_filter_units"]].describe()


,bottle_units,pitcher_units,total_bottle_filters,pitcher_filter_units
count,16688.000000,16688.000000,16688.000000,16688.000000
mean,0.549137,0.305669,2.998082,1.950803
std,1.215294,0.496755,4.365582,3.708167
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,1.000000,0.000000
75%,1.000000,1.000000,5.000000,2.000000
max,70.000000,10.000000,148.000000,48.000000


In [32]:
order_level.to_csv("../data/processed/order_breakdown.csv")